### 配置Langchain

In [32]:
#示例：langgraph_hello.py
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
# pip install langgraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("API_KEY")

print("API_KEY:", API_KEY)

from langchain.schema import HumanMessage, SystemMessage
# 创建智谱AI LLM实例
llm = ChatOpenAI(
    temperature=0.0,
    model="glm-4",
    openai_api_key=API_KEY,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

print(llm.openai_api_key)


API_KEY: f532b4bd71324c1ca2fd7e22d4eb41da.4Sfpr8wE6qEVV5BY
**********


#### 提示词模板、输出转换

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有帮助的助手。"),
    ("user", "请根据以下内容回答问题：{input}"),
])

output_parser = StrOutputParser()
chain = prompt|llm|output_parser

chain.invoke({"input": "生成一个20字左右的小诗歌"})

'春风拂面花千树，\n燕舞蓝天梦无边。\n美好时光手中握，\n共享人间四月天。'

In [ ]:
prompt_template_jokes = ChatPromptTemplate.from_template(
    "给我讲一个关于{topic}的{style}笑话，要求笑话内容不超过50字，只能由你来原创，不能抄已有的。"
)


result = prompt_template_jokes.format_messages(topic="浙江大学软件学院", style="冷")
print(result[0].content)
chain.invoke(result[0].content)

给我讲一个关于浙江大学软件学院的冷笑话，要求笑话内容不超过50字，只能由你来原创，不能抄已有的。


'浙软学子编程强，\n写出代码无人详，\n问其原因何其长？\n答曰：我是浙大软件人，代码自己会解释。'

#### 工作流LCEL

In [ ]:
@tool
def search(query: str):
    """搜索工具"""
    return f"搜索结果：关于{query}的回答是...（模拟搜索结果）"

tools = [search]

tool_node = ToolNode(tools)

# bind_tools方法将工具绑定到ToolNode
model = llm.bind_tools(tools)

# 是否继续，只允许返回"tools"或END
def should_continue(state: MessagesState) -> Literal["tools", END]:
    """判断是否继续"""
    messages = state["messages"]
    last_message = messages[-1]
    # 如果llm调用了工具，就转到工具节点
    if last_message.tool_calls:
        return "tools"
    # 否则结束，回复用户
    return END

# 调用函数
def call_model(state:MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    # 返回一个消息列表，用于添加到消息列表中
    return {"messages": [response]}

# 初始化状态图，定义一个新的状态图
workflow= StateGraph(MessagesState)

workflow.add_node("agent",call_model) 
workflow.add_node("tools", tool_node) 

workflow.